In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from  sklearn.model_selection import train_test_split as tts
from itertools import count
import numpy.linalg as la
import random

In [2]:
data1 = pd.read_csv('mnist_train.csv')
data2 = pd.read_csv('mnist_test.csv')

labels1 = np.asfarray(data1.iloc[:,data1.columns == 'label'])
images1 = np.asfarray(data1.iloc[:,data1.columns != 'label'])

labels2 = np.asfarray(data2.loc[:,data2.columns == 'label'])
images2 = np.asfarray(data2.loc[:,data2.columns != 'label'])
# Xtrain, Xtest, ytrain, ytest = tts(images, labels, test_size = 0.001)
Xtrain, Xtest, ytrain, ytest = images1, images2, labels1, labels2

# QDA on MultiClass

## 1.Fitting Classifier

In [618]:
x_train_class = [] ; size_train_class = [] ; y_train_class =[];
cov_class = []; mean_class = []
# u = np.zeros((10,784,784)); 
first_term = np.zeros((10));
third_term = np.zeros((10));
        
for k in range(10):
    tmp = Xtrain[[i for i, j in zip(count(), ytrain) if j == k],:]
    x_train_class.append(tmp)
    y_train_class.append(np.ones((len(tmp),1))*k)
#     print(len(tmp[0]))
#     print(len(x_train_class[0][0]))
    size_train_class.append(len(tmp))
    cov_class.append(np.cov(tmp.T))
    mean_class.append(np.mean(tmp,axis = 0))
#     u[k], s, vh = la.svd(cov_class[k])
#     S = np.diag(s)
#     s_invhalf = np.sqrt(la.inv(S))
    first_term[k] = -1*(np.log(np.sqrt(la.norm(cov_class[k]))))
    third_term[k] = np.log(size_train_class[k]/ytrain.shape[0])


## 2.Prediction

In [619]:
def predict1(x):
    delta = []
    for k in range(10):
        value = np.array([x - mean_class[k]])
        tmp = (-1/2)*np.dot(value,np.dot(la.pinv(cov_class[k]),(value.T)))
        delta.append((first_term[k] + tmp + third_term[k]))
    return np.argmax(delta)

## 3.Accuracy

In [620]:
accu = 0
preds = np.zeros((5,1))
for j in range(5):
    rand_index = random.randint(1,9989)
    k = 0
    for i in range(rand_index,5+rand_index):
        preds[k] = predict(Xtest[i])
        print(i,j)
        k +=1
    accu += (preds==ytest[rand_index:5+rand_index]).mean()


5784 0
5785 0
5786 0
5787 0
5788 0
4216 1
4217 1
4218 1
4219 1
4220 1
1647 2
1648 2
1649 2
1650 2
1651 2
1014 3
1015 3
1016 3
1017 3
1018 3
9466 4
9467 4
9468 4
9469 4
9470 4


In [621]:
print("Accuracy on Test Dataset :",(accu)/5*100,"%")

Accuracy on Test Dataset : 80.0 %


# LDA on Multiclass : $\Sigma$ is common = I

In [622]:
mean_class_ldamulti = []
u = np.zeros((10,784,784)); 
second_term = np.zeros((10));
    
for k in range(10):
    tmp = x_train_class[k]

    mean_class_ldamulti.append(np.mean(tmp,axis = 0))
    second_term[k] = np.log(size_train_class[k]/ytrain.shape[0])


In [623]:
def predict2(x):
    delta = []
    for k in range(10):
        value = np.array([x - mean_class_ldamulti[k]])
        tmp = (-1/2)*np.square(la.norm(value))
        delta.append((tmp + second_term[k]))
    return np.argmax(delta)

In [624]:
accu1 = 0
accu2 = 0
preds1 = np.zeros((5,1))
preds2 = np.zeros((5,1))
for j in range(5):
    rand_index = random.randint(1,9989)
    k = 0
    for i in range(rand_index,5+rand_index):
        preds1[k] = predict1(Xtest[i])
        preds2[k] = predict2(Xtest[i])
#         print(i,j)
        k +=1
        
    accu1 += (preds1==ytest[rand_index:5+rand_index]).mean()
    accu2 += (preds2==ytest[rand_index:5+rand_index]).mean()

print('QDA -->Accuracy on Test dataset:',(accu1/5)*100)
print('LDA -->Accuracy on Test dataset:',(accu2/5)*100)

QDA -->Accuracy on Test dataset: 87.99999999999999
LDA -->Accuracy on Test dataset: 80.0


# LDA on Binary Class

In [625]:
mean = []
term = np.zeros((2));

        
for k in range(2):
    tmp = x_train_class[k]
    cov_class.append(np.cov(tmp.T))
    mean_class.append(np.mean(tmp,axis = 0))
    
    term[k] = np.log(size_train_class[k]/ytrain.shape[0])

cov = (cov_class[0] + cov_class[1])/2

In [626]:
def predict3(x):
    delta = []
    for k in range(2):
        value = np.array([x - mean_class[k]])
        tmp = (-1/2)*np.dot(value,np.dot(la.pinv(cov),(value.T)))
        delta.append((tmp + term[k]))
    return np.argmax(delta)

In [627]:
Xtest_b = Xtest[[i for i, j in zip(count(), ytest) if (j == 0 or j==1)],:]
ytest_b = ytest[[i for i, j in zip(count(), ytest) if (j == 0 or j==1)],:]
ytest_b.shape

(2115, 1)

In [628]:
accu_b = 0
preds_b = np.zeros((200,1))

for i in range(200):
    preds_b[i] = predict3(Xtest_b[i])
#     print(i)

In [629]:
accu_b = (preds_b[:200]==ytest_b[:200]).mean()
print('LDA in Binary Classification -->Accuracy on Test dataset:',(accu_b)*100)

LDA in Binary Classification -->Accuracy on Test dataset: 99.5
